# Feature Importance dos Autovalores do Kapetanovic

Nesse jupyter, o objeitivo é verificar quantos autovalores daquela mariz R a gnt pode usar pra fazer a detecção com machine learning. Nós começamos fazendo a razão entre o maior e o segundo maior autovalor. Depois, passamos de uma feature (razão) para duas (os dois maiores autovalores em si, sem fazer a razão). A detecção melhorou! Pois bem, então vamos usar mais autovalores né? Coloca o terceiro maior ai como feature então. Beleza, parece que melhorou mais um tótózinho. Então decidimos extrair TODOS os autovalores daquela matriz, e agora vamos ver até qual a gnt vai usar.

O dataset que a gnt vai usar pra verificar isso é composto por 256 antenas, 128 símbolos e 1 usuário conectado. SNR de -10 até 30 dB, passo 1. Potência do espião indo de 0 até 2.5, passo 0.5. Cada cenário tem repetibilidade 100. Como temos 128 símbolos, então são 128 autovalores na matriz R, logo, temos 128 + 3 features (autovalores + snr + E + eta). O E e o eta do Hassan vão ficar por ai msm pq a gnt já ta ligado que elas são as melhores features.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from time import sleep, time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

In [ ]:
# SO COMECA QUANDO O ALGORITMO DE GERAR OS DATASET TERMINAR
while os.path.isfile("../../CSV/dataset_256usuarios_256antenas_128simbolos_autovalores_completos.csv") == False:
    sleep(1 * 60)

In [ ]:
#DEFININDO ALGUNS PARÂMETROS DO GRÁFICO
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
params = {
    'figure.figsize': [10, 5], 
    'axes.labelsize': 18,
    'axes.titlesize':20, 
    'font.size': 18,
    'legend.fontsize': 14, 
    'xtick.labelsize': 12, 
    'ytick.labelsize': 12,
    'axes.axisbelow': True
}
plt.rcParams.update(params)

## Definição de Funções

In [ ]:
def treinarEClassificar(xTrain, xTest, yTrain, yTest, classificador):
    
    # INSTANCIANDO O CLASSIFICADOR
    objClassificador = eval(classificador)
    
    # TREINANDO
    objClassificador.fit(xTrain, yTrain)
    
    # PREDIZENDO
    yPred = objClassificador.predict(xTest)
    
    # RETORNANDO A ACURACIA
    return accuracy_score(yTest, yPred)

In [ ]:
def salvarResultado(arquivoSalvar, nomeDataset, arrayAcuraciaMedia, arrayDesvioPadrao):
    objFile = open(arquivoSalvar, "a")
    objFile.write("Dataset: " + nomeDataset)
    objFile.write("\narrayAcuraciaMediaCadaQtdAutovaloresIgnorados = " + str(arrayAcuraciaMedia))
    objFile.write("\narrayDesvioPadraoAcuraciaCadaQtdAutovaloresIgnorados = " + str(arrayDesvioPadrao) + "\n\n")
    objFile.close()

## Parâmetros Iniciais

In [ ]:
# TEMOS DATASETES COM QUAIS QTDs DE USUARIOS?
rangeQtdUsuarios = [1,2,4,8,16,32,64,128,256]

# CLASSIFICADOR PRA USAR NA AVALIACAO
classificador = "RandomForestClassifier(n_estimators=10, n_jobs=-1)"

# QTD DE ITERACOES DO K FOLD    
qtdFoders=5

# ARQUIVO DE SALVAR OS RESULTADOS
arquivoSalvar = "../../Resultados/featureImportanceAutovalores"+str(time())+".txt"

## Feature Importance

In [ ]:
for qtdUsuariosAtual in rangeQtdUsuarios:
    
    # ABRINDO
    caminhoCSV = "../../CSV/dataset_" + str(qtdUsuariosAtual) + "usuarios_256antenas_128simbolos_autovalores_completos.csv"
    dataframe = pd.read_csv(caminhoCSV)  

    # SEPARANDO DATA, TARGET E FEATURES
    data    = dataframe.iloc[:, 0:-2].to_numpy()
    target  = dataframe.iloc[:, -1].to_numpy()
    colunas = list(dataframe.columns[:-2])

    # OBTENDO O VETOR DE IMPORTANCIA DAS FEATURES
    objDT = DecisionTreeClassifier()
    objDT.fit(data, target)
    arrayImportancias = objDT.feature_importances_

    # ORDENANDO DA MENOS IMPORTANTE PRA MAIS IMPORTANTE
    arrayImportancias, colunasOrdenadas = zip(*sorted(zip(arrayImportancias, colunas)))

    # PLOTANDO AS limite PRIMEIRAS
    limite = 10
    plt.bar(np.flip(colunasOrdenadas)[:limite], np.flip(arrayImportancias)[:limite])
    plt.title("Feature Importance - Usuários: "+str(qtdUsuariosAtual))
    plt.yscale("log")
    plt.xticks(rotation=45)
    plt.grid(alpha=0.5)
    plt.show()

## Abandonando os autovalores um por um

In [ ]:
# MATRIZES QUE VAO GUARDAR OS ARRAYS DE ACURACIA MEDIA E DESVIO DE CADA QTD DE USUARIOS
matrizAcuraciasMediasCadaQtdUsuarios = []
matrizDesviosCadaQtdUsuarios         = []

# CADA QTD DE USUARIOS
for qtdUsuariosAtual in rangeQtdUsuarios:
    
    # ABRINDO
    caminhoCSV = "../../CSV/dataset_" + str(qtdUsuariosAtual) + "usuarios_256antenas_128simbolos_autovalores_completos.csv"
    dataframe = pd.read_csv(caminhoCSV)  

    # SEPARANDO DATA, TARGET E FEATURES
    data    = dataframe.iloc[:, 0:-2].to_numpy()
    target  = dataframe.iloc[:, -1].to_numpy()
    colunas = list(dataframe.columns[:-2])
    
#     # REDUZINDO O DATASET PARA TESTES
#     data, target = shuffle(data, target)
#     data   = data[:int(len(data)/100)] 
#     target = target[:int(len(target)/100)]
    
    # CRIANDO OS ARRAYS DE ACURACIA MEDIA E DESVIO QUE VAO GUARDAR EM CADA POSICAO O RESULTADO PARA A QTD DE AUTOVALORES IGNORADOS
    arrayAcuraciaMedia = []
    arrayDesvioPadrao  = []
    
    # DROPANDO UMA POR UMA DE CADA VEZ
    print("Usuários: "+str(qtdUsuariosAtual), end=" - Qtd features ignoradas: ")
    for i in range(1, len(colunas) - 2):
        
        # REDUZINDO A QTD DE FEATURES, DROPANDO COLUNA POR COLUNA
        data = np.delete(data, len(data[0])-1, axis=1)
        print(data.shape[1], end=" ")
        
        # ARRAY PRA GUARDAR AS ACURACIAS DOS KFOLDS DESSA QTD DE FEATURES
        acuraciasQtdFeaturesAtual = []
    
        # FAZENDO UM KFOLD
        objKFold = KFold(n_splits=qtdFoders, shuffle=True)
        for trainIndex, testIndex in objKFold.split(data):
            
            # SEPARANDO TREINAMENTO E TESTE
            xTrain, xTest, yTrain, yTest = data[trainIndex], data[testIndex], target[trainIndex], target[testIndex]
            
            # OBTENDO A ACURACIA DA ITERACAO ATUAL DO K FOLD
            acuraciasQtdFeaturesAtual.append(treinarEClassificar(xTrain, xTest, yTrain, yTest, classificador))
            
        # AGORA QUE TERMINOU O KFOLD PARA A QTD DE AUTOVALORES IGNORADOS DESSA ITERACAO, VOU GUARDAR A MEDIA E O DESVIO DA ACURACIA
        arrayAcuraciaMedia.append(np.mean(acuraciasQtdFeaturesAtual))
        arrayDesvioPadrao.append(np.std(acuraciasQtdFeaturesAtual))
        
    # COLOCANDO NA MATRIZ PRA PLOTAR DEPOIS E SALVANDO O RESULTADO
    salvarResultado(arquivoSalvar, caminhoCSV, arrayAcuraciaMedia, arrayDesvioPadrao)
    matrizAcuraciasMediasCadaQtdUsuarios.append(arrayAcuraciaMedia)
    matrizDesviosCadaQtdUsuarios.append(arrayDesvioPadrao)

#### Printando

In [ ]:
for i, qtdUsuariosAtual in enumerate(rangeQtdUsuarios):
    
    arrayAcuraciaMedia = matrizAcuraciasMediasCadaQtdUsuarios[i]
    arrayDesvioPadrao  = matrizDesviosCadaQtdUsuarios[i]
    
    plt.errorbar(x=np.arange(1,len(arrayAcuraciaMedia)+1), y=arrayAcuraciaMedia, yerr=arrayDesvioPadrao)
    plt.title("Usuários: "+str(qtdUsuariosAtual))
    plt.xlabel("Qtd de autovalores ignorados")
    plt.ylabel("Acurácia")
    plt.xlim(0,129)
    plt.ylim(0.999*(min(arrayAcuraciaMedia)-max(arrayDesvioPadrao)),1.001*(max(arrayAcuraciaMedia)+max(arrayDesvioPadrao)))
    plt.grid(alpha=0.5)
    plt.show()